In [1]:
import decisiontree_gpu as dtree
import numpy as np
import time

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("DTREE-SUSY").config("spark.executor.memory", "8gb").getOrCreate()
sc = spark.sparkContext

In [2]:
# 1. Leemos el dataset
rdd = sc.textFile('../datasets/SUSY.csv')
def append_ret(a,b):
    a.append(b)
    return a
# 2. Separamos y ponemos la etiqueta al final en vez de al principio
rdd = rdd.map(lambda line: line.split(','))
rdd = rdd.map(lambda line: append_ret(line[1:], line[0]))
# 3. Subdividimos el RDD en Split de Training (90 %) y Split de Test (10 %)
(trainingData, testData) = rdd.randomSplit([0.9, 0.1], seed=35)

In [3]:
# 4. Damos el número de particiones deseadas en el RDD para trabajar con el modelo
trainingData = trainingData.repartition(225)

In [4]:
def gpu_work(max_depth=6, min_samples_per_node=1, tpb=128):
    def _gpu_work(data):
        # 1. Procesamos el dataset
        inp = np.asarray(list(data), dtype=np.float32)
        return [dtree.train_tree(inp, max_depth, min_samples_per_node, tpb)]    
    return _gpu_work

In [5]:
# 5. Ejecutamos el modelo sobre cada una de las particiones
inicio = time.time()
arboles = trainingData.mapPartitions(gpu_work(max_depth=7))
arboles = arboles.collect()
fin = time.time()
print('Tiempo utilizado:' fin-inicio)

In [6]:
def predict(sample, trees):
    counter = 0
    for tree in trees:
        counter += dtree.predict(sample, tree)
    
    return 1 if counter > len(trees) / 2 else 0

def evaluar(dataset, trees):
    aciertos = 0
    for sample in dataset:
        etiqueta = predict(sample, trees)
        if int(etiqueta) == int(sample[-1]):
            aciertos += 1
        
    return aciertos / dataset.shape[0]


In [7]:
test = np.float32(testData.collect())

In [8]:
evaluar(test, arboles)

0.7873251236156349

In [ ]:
"""
PENDIENTE: EVALUAR SPARK
VALIDACIÓN CRUZADA
"""